# TO START THE GAME, PRESS: Kernal > Restart & Run All

In [1]:
import collections
import copy
import pytest
from typing import List, Optional, Tuple

In [2]:
class Position(collections.namedtuple('Position', 'x, y')):
    """ Simple 2d coordinate with simple arithmetic """
    def __add__(self, other: 'Position'):
        return Position(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other: 'Position'):
        return Position(self.x - other.x, self.y - other.y)
    
    @property
    def right(self):
        return Position(self.x + 1, self.y)
    @property
    def left(self):
        return Position(self.x - 1, self.y)
    @property
    def up(self):
        return Position(self.x, self.y + 1)
    @property
    def down(self):
        return Position(self.x, self.y - 1)

# Test
assert Position(1, 0) + Position(1, 2) == Position(2, 2)
assert Position(1, 0) - Position(1, 2) == Position(0, -2)
assert Position(0, 0).right == Position(1, 0)
assert Position(0, 0).left == Position(-1, 0)
assert Position(0, 0).up == Position(0, 1)
assert Position(0, 0).down == Position(0, -1)

In [3]:
class GameObject:
    """ mother of all game objects """
    @property
    def position(self):
        return Position(self.x, self.y)
    
    @position.setter
    def position(self, value: Position):
        (self.x, self.y) = value

In [4]:
class Player(GameObject):
    """ Its'a me! """
    text_sprite = "P"
    
    def __init__(self, x: int, y: int):
        self.x, self.y = x, y
        self.in_portal = False
    
    def __repr__(self):
        return f'<Player({self.x}, {self.y}): in_portal={self.in_portal}>'

# Tests
player = Player(0, 1)
assert player.position == (player.x, player.y) == (0, 1)
player.position = (0, 2)
assert player.position == (player.x, player.y) == (0, 2)

In [5]:
class Block(GameObject):
    """ Movable block """
    text_sprite = "B"

    def __init__(self, x: int, y: int):
        self.x, self.y = x, y
        self.is_pushable = True
        self.in_portal = False

    def __repr__(self):
        return f'<Block({self.x}, {self.y}): in_portal={self.in_portal}>'

# Tests
block = Block(0, 1)
assert block.position == (block.x, block.y) == (0, 1)
assert block.is_pushable == True
assert block.in_portal == False
block.position = (0, 2)
assert block.position == (block.x, block.y) == (0, 2)

In [6]:
class Wall(GameObject):
    """ Immovable wall """
    text_sprite = "W"

    def __init__(self, x: int, y: int):
        self.x, self.y = x, y
        self.is_pushable = False
        self.in_portal = False

    def __repr__(self):
        return f'<Block({self.x}, {self.y}): in_portal={self.in_portal}>'

# Tests
wall = Wall(0, 1)
assert wall.position == (wall.x, wall.y) == (0, 1)
assert wall.is_pushable == False
assert wall.in_portal == False
wall.position = (0, 2)
assert wall.position == (wall.x, wall.y) == (0, 2)

In [7]:
class Level(List[GameObject]):
    """ Holds all data for a single level. """
    def __init__(self, game_objects: Optional[List[GameObject]]=None, height=10, width=30):
        if game_objects:
            self.extend(game_objects)
        self.height = height
        self.width = width

    @property
    def player(self):
        """ Register player if exists """
        return next((obj for obj in self if isinstance(obj, Player)), None)

# Test basics
assert len(Level()) == 0
assert Level(height=1).height == 1
assert Level(width=2).width == 2

## should register player if exists
player = Player(0, 0)
assert Level([player]).player == player

In [8]:
def render(level: Level):
    """ Render a level consisting of gameobjects into text.
        Note that this renders y coordinate backwards.
        (Think first quadrant.)
    """
    def is_out_of_bound(position: Position) -> bool:
        return position.x < 0 or position.y < 0 or position.x >= level.width or position.y >= level.height

    level_drawn = [[" " for _ in range(level.width)] for __ in range(level.height)]
    for thing in level:
        if is_out_of_bound(thing.position):
            # Don't render things that are out of bounds
            continue
        level_drawn[thing.y][thing.x] = thing.text_sprite
    level_drawn = reversed(level_drawn) # Draw y coord backwards!
    return '\n'.join([''.join(row) for row in level_drawn])

Level.render = render

# Test `render()`
assert Level([Player(0, 1), Block(1, 1), Block(1, 2)], height=4, width=4).render() == (
    "    \n"
    " B  \n"
    "PB  \n"
    "    ")

## Test rendering out of bounds (they shouldn't show up)
assert Level([Block(-1, 0), Block(0, 2), Block(2, 1), Block(1, -1)], height=2, width=2).render() == "  \n  "

In [9]:
def get_object_in_position(level: Level, position: Position):
    """ Gets the first game object in position. Returns None if not found. """
    for thing in level:
        if thing.position == position:
            return thing
Level.get = get_object_in_position

# Test `get_object_in_position(Position)`
p = Player(0, 0)
b = Block(0, 1)
b2 = Block(0, 1) # Shadowed
w = Wall(1, 0)

level = Level([p, b, w])
assert level.get(Position(0, 0)) == p
assert level.get(Position(0, 1)) == b
assert level.get(Position(1, 0)) == w
assert level.get(Position(1, 1)) == None

In [10]:
class NoPlayerException(Exception):
    """ There is no player in this level! """

def press(level: Level, button: str):
    """ Push that button! Main game logic is inside this function """
    if not level.player:
        raise NoPlayerException('there is no player in this level, stupid')
    player = level.player
    
    # Todo: Refactor this mess. They all follow the same logic.
    if button == 'right':
        thing_in_front = level.get(player.position.right)
        if thing_in_front is None:
            # Nothing is in front! Free to go!
            player.x += 1
        elif thing_in_front.is_pushable:
            # Something's in front but looks pushable
            if level.get(thing_in_front.position.right) is None:
                # Hey I can push this thing!
                thing_in_front.x += 1
                player.x += 1
                
    elif button == 'left':
        thing_in_front = level.get(player.position.left)
        if thing_in_front is None:
            player.x -= 1
        elif thing_in_front.is_pushable:
            if level.get(thing_in_front.position.left) is None:
                thing_in_front.x -= 1
                player.x -= 1
                
    elif button == 'up':
        thing_in_front = level.get(player.position.up)
        if thing_in_front is None:
            player.y += 1
        elif thing_in_front.is_pushable:
            if level.get(thing_in_front.position.up) is None:
                thing_in_front.y += 1
                player.y += 1
                
    elif button == 'down':
        thing_in_front = level.get(player.position.down)
        if thing_in_front is None:
            player.y -= 1
        elif thing_in_front.is_pushable:
            if level.get(thing_in_front.position.down) is None:
                thing_in_front.y -= 1
                player.y -= 1
    else:
        raise NotImplementedError(f'?? what is "{button}"?')
Level.press = press

# Test moving without obstacles
level = Level([Player(0, 0)])

assert level.press('right') or  level.player.position == (1, 0)
assert level.press('up')    or  level.player.position == (1, 1)
assert level.press('left')  or  level.player.position == (0, 1)
assert level.press('down')  or  level.player.position == (0, 0)

with pytest.raises(NotImplementedError):
    level.press('space')

# Test moving into walls

#  W
# WPW  < I can't move!! help!!1!1
#  W
player = Player(1, 1)
walls = [Wall(0, 1), Wall(2, 1), Wall(1, 0), Wall(1, 2)]
level = Level([player] + walls)
level.press('right')
assert player.position == (1, 1)
level.press('left')
assert player.position == (1, 1)
level.press('up')
assert player.position == (1, 1)
level.press('down')
assert player.position == (1, 1)

# Test moving blocks

#
#  B
# BPB
#  B
#
player = Player(2, 2)
blocks = [Block(1, 2), Block(3, 2), Block(2, 1), Block(2, 3)]
level = Level([player] + blocks)
level.press('right')
assert player.position == (3, 2)
assert isinstance(level.get((4, 2)), Block)
level.press('left')
level.press('up')
assert player.position == (2, 3)
assert isinstance(level.get((2, 4)), Block)
level.press('down')
level.press('left')
assert player.position == (1, 2)
assert isinstance(level.get((0, 2)), Block)
level.press('right')
level.press('down')
assert player.position == (2, 1)
assert isinstance(level.get((2, 0)), Block)
level.press('up') # Just for symmetry's sake

# Test raise if no player
with pytest.raises(NoPlayerException):
    Level().press('right')

# Main game

In [11]:
# Level Setup
map_height = 10
map_width = 30

player = Player(5, 5)
walls = [Wall(n, 0) for n in range(map_width)] + [Wall(n, map_height-1) for n in range(map_width)] \
      + [Wall(0, n) for n in range(1, map_height-1)] + [Wall(map_width-1, n) for n in range(1, map_height-1)]

block = Block(2, 4)
level = Level([])
level.append(player)
level.append(block)
level.extend(walls)

# Level history for undo feature
level_history = [level]

print(level.render())

WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
W                            W
W                            W
W                            W
W    P                       W
W B                          W
W                            W
W                            W
W                            W
WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW


In [12]:
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, HBox, VBox

empty_button = Button(layout=Layout(width='30px'))
undo_button = Button(icon='history', layout=Layout(width='30px'))
up_button = Button(icon='arrow-up', layout=Layout(width='30px'))
down_button = Button(icon='arrow-down', layout=Layout(width='30px'))
left_button = Button(icon='arrow-left', layout=Layout(width='30px'))
right_button = Button(icon='arrow-right', layout=Layout(width='30px'))

display(HBox([
    VBox([undo_button, left_button]),
    VBox([up_button, down_button]),
    VBox([empty_button, right_button])]))

def directional_button_handler(direction: str):
    """ Button handler for directional buttons. History is saved here. """
    def handler(b):
        global level
        level = copy.deepcopy(level)
        level.press(direction)
        level_history.append(level)
    return handler

def undo_button_handler(b):
    """ Undo stuff """
    global level
    if not len(level_history):
        raise Exception('This is your first move, dumbass.')
    level_history.pop()
    level = level_history[-1]

def after_button_click(b):
    clear_output()
    print(level.render())

# Register button handlers
up_button.on_click(directional_button_handler('up'))
down_button.on_click(directional_button_handler('down'))
left_button.on_click(directional_button_handler('left'))
right_button.on_click(directional_button_handler('right'))
undo_button.on_click(undo_button_handler)

# Register button handler for rendering.
# Must be registered after everything else.
up_button.on_click(after_button_click)
down_button.on_click(after_button_click)
left_button.on_click(after_button_click)
right_button.on_click(after_button_click)
undo_button.on_click(after_button_click)

# Initial render
print(level.render())

WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
W                            W
W                            W
W       PB                   W
W                            W
W                            W
W                            W
W                            W
W                            W
WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
